# Plots for paper

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
try:
    sys.path.remove("../py_src") # Hacky way to add higher directory to python modules path. 
except:
    pass
sys.path.append("../py_src") # Means that I dont have to make src/ a proper python package

In [34]:

import matplotlib.pyplot as plt 

import numpy as np 
import json
import pandas as pd 
import corner
import scienceplots


def load_data_and_check_for_weird_values(path):

    df_posterior = pd.read_parquet(path)  

    extra_median = df_posterior['psi_gw'].median()
    if extra_median < 2.4: #psi is weird / not converged so don't plot it 
        #print("Ignoring this file since psi not converged",extra_median)
        return_code = 1
    else:
        return_code = 0

    return return_code


def load_data(path,variables_to_plot):

    df_posterior = pd.read_parquet(path)  
    return df_posterior[variables_to_plot]

In [30]:
import glob 
list_of_files = sorted(glob.glob("../data/nested_sampling/data_for_manuscript/compressed_files/*shifted_prior*.parquet.gzip"))
parameters = ["omega_gw","phi0_gw","psi_gw","iota_gw","delta_gw","alpha_gw","h",]

list_of_well_behaved_files = []

for f in list_of_files:
    return_code = load_data_and_check_for_weird_values(f)

    if return_code == 0:
        list_of_well_behaved_files.extend([f])


#f = list_of_files[1]
#df_reference,return_code = load_data(f,parameters)

In [31]:
len(list_of_well_behaved_files) / len(list_of_files)

0.8792682926829268

In [37]:
f = list_of_well_behaved_files[0]
df_reference = load_data(f,parameters)

In [51]:
import sys 


wasserstein_results = np.zeros((len(list_of_well_behaved_files)-1,len(parameters)))
for i in range(1,len(list_of_well_behaved_files)):
    f = list_of_well_behaved_files[i]
    df = load_data(f,parameters)
    k = 0
    for p in parameters:
    
        w = wasserstein_distance(df_reference[p].to_numpy(),df[p].to_numpy())
        wasserstein_results[i-1,k] += w 
        k += 1


    

In [52]:
wasserstein_results

array([[1.15167442e-11, 3.22140537e-03, 8.10353933e-04, ...,
        5.17258403e-04, 5.46483693e-04, 1.18632445e-14],
       [4.25041766e-12, 7.78458356e-04, 1.41921006e-03, ...,
        8.37109855e-04, 5.48179522e-04, 2.84119237e-14],
       [3.16225534e-11, 5.81617021e-03, 2.84747361e-03, ...,
        3.64492445e-03, 2.16325591e-03, 8.73393303e-14],
       ...,
       [3.48965409e-11, 7.73209964e-03, 2.09258928e-04, ...,
        1.45413046e-03, 3.62032547e-04, 5.19022783e-14],
       [2.12230551e-11, 5.68820912e-03, 7.88652348e-04, ...,
        1.65491486e-04, 5.57112803e-04, 1.98170987e-13],
       [6.45573113e-12, 1.99853415e-03, 2.42366936e-04, ...,
        1.40261766e-03, 3.89908483e-04, 1.50689352e-14]])

In [56]:
np.median(wasserstein_results,axis=0)

array([1.48691908e-11, 3.25747447e-03, 1.23656422e-03, 1.21191383e-01,
       7.92314733e-04, 1.11939380e-03, 1.00464192e-13])

In [58]:
injection_parameters = [5e-7,0.20,2.50,1.0,1.0,1.0,1e-12]
np.median(wasserstein_results,axis=0) / injection_parameters


array([2.97383815e-05, 1.62873723e-02, 4.94625687e-04, 1.21191383e-01,
       7.92314733e-04, 1.11939380e-03, 1.00464192e-01])

In [38]:
df = load_data(f,parameters)

In [39]:
df

,omega_gw,phi0_gw,psi_gw,iota_gw,delta_gw,alpha_gw,h
0,5.000001e-07,0.193300,2.595132,0.758582,1.001830,1.038984,1.062884e-12
1,4.999990e-07,0.195338,2.594946,0.914982,1.001852,1.038783,1.183517e-12
2,5.000085e-07,0.195745,2.596255,0.743157,1.002767,1.039714,1.052433e-12
3,5.000015e-07,0.195423,2.596508,0.827010,1.002305,1.039820,1.114980e-12
4,4.999917e-07,0.192893,2.595399,0.968847,1.001697,1.039041,1.229464e-12
...,...,...,...,...,...,...,...
11605,5.000000e-07,0.194168,2.595424,1.013730,1.001215,1.038944,1.270089e-12
11606,4.999969e-07,0.193350,2.596211,0.977245,1.002433,1.039815,1.236280e-12
11607,4.999981e-07,0.193549,2.596407,0.939210,1.002701,1.039909,1.204758e-12
11608,4.999961e-07,0.194011,2.595116,0.887927,1.001503,1.039113,1.161320e-12


In [ ]:
ws_df = df.groupby('grp').apply(lambda x: stats.wasserstein_distance(x['p'], x['q']))


In [17]:
from scipy.stats import wasserstein_distance

var = "h"
wasserstein_distance(df_reference[var].to_numpy(),df[var].to_numpy())



1.1943624520184697e-13